In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import Model


2023-06-16 01:24:32.720213: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-16 01:24:32.758459: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-16 01:24:32.759551: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 01:24:33.656359: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
import os
import sys
root = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(root)

In [8]:
# import tensorflow as tf
# from tensorflow.keras.layers import Model, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from asynfed.client.frameworks.tensorflow.tensorflow_sequential_model import TensorflowSequentialModel

class VGG16(TensorflowSequentialModel):
# class VGG16(Model):
    def __init__(self, input_features=(32, 32, 3), output_features=10):
        super().__init__(input_features= input_features, output_features= output_features)

    def create_model(self, input_features, output_features):
        # First block
        self.conv1_1 = Conv2D(64, (3, 3), activation='relu', padding='same', input_shape= input_features)
        self.conv1_2 = Conv2D(64, (3, 3), activation='relu', padding='same')
        self.pool1 = MaxPooling2D((2, 2))

        # Second block
        self.conv2_1 = Conv2D(128, (3, 3), activation='relu', padding='same')
        self.conv2_2 = Conv2D(128, (3, 3), activation='relu', padding='same')
        self.pool2 = MaxPooling2D((2, 2))

        # Remaining blocks... 

        # Third block
        self.conv3_1 = Conv2D(256, (3, 3), activation='relu', padding='same')
        self.conv3_2 = Conv2D(256, (3, 3), activation='relu', padding='same')
        self.conv3_3 = Conv2D(256, (3, 3), activation='relu', padding='same')
        self.pool3 = MaxPooling2D((2, 2))

        # Fourth block
        self.conv4_1 = Conv2D(512, (3, 3), activation='relu', padding='same')
        self.conv4_2 = Conv2D(512, (3, 3), activation='relu', padding='same')
        self.conv4_3 = Conv2D(512, (3, 3), activation='relu', padding='same')
        self.pool4 = MaxPooling2D((2, 2))

        # Fifth block
        self.conv5_1 = Conv2D(512, (3, 3), activation='relu', padding='same')
        self.conv5_2 = Conv2D(512, (3, 3), activation='relu', padding='same')
        self.conv5_3 = Conv2D(512, (3, 3), activation='relu', padding='same')
        self.pool5 = MaxPooling2D((2, 2))

        # Dense layers
        self.flatten = Flatten()
        self.fc1 = Dense(4096, activation='relu')
        self.fc2 = Dense(4096, activation='relu')
        self.fc3 = Dense(output_features, activation='softmax')
        
        # Dense layers
        self.flatten = Flatten()
        self.fc1 = Dense(512, activation='relu')
        # randomly drop some nodes
        self.dropout1 = Dropout(0.4)
        self.fc2 = Dense(256, activation='relu')
        # randomly drop some nodes
        self.dropout2 = Dropout(0.4)
        self.fc3 = Dense(output_features, activation='softmax')
        

    def call(self, x):
        # first block
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)

        # second block
        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)

        # third block
        x = self.conv3_1(x)
        x = self.conv3_2(x)
        x = self.conv3_3(x)
        x = self.pool3(x)

        # fourth block
        x = self.conv4_1(x)
        x = self.conv4_2(x)
        x = self.conv4_3(x)
        x = self.pool5(x)

        # fifth block
        x = self.conv5_1(x)
        x = self.conv5_2(x)
        x = self.conv5_3(x)
        x = self.pool5(x)

        # dense block
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        
        return self.fc3(x)

    def create_loss_object(self):
        return tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    def create_optimizer(self):
        return tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

    def create_train_metric(self):
        return tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy'), tf.keras.metrics.Mean(name='train_loss')

    def create_test_metric(self):
        return tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy'), tf.keras.metrics.Mean(name='test_loss')

    def get_train_performance(self):
        return float(self.train_performance.result())

    def get_train_loss(self):
        return float(self.train_loss.result())
    
    def get_test_performance(self):
        return float(self.test_performance.result())

    def get_test_loss(self):
        return float(self.test_loss.result())


In [9]:
model = VGG16()

2023-06-16 01:26:59.204604: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-16 01:26:59.205619: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
model.optimizer.learning_rate

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.001>

In [ ]:
# model = CustomVGG16Model(10)
# model.compile(optimizer=optimizers.SGD(learning_rate=0.001, momentum=0.9), 
#               loss='categorical_crossentropy', 
#               metrics=['accuracy'])

# Learning rate scheduler function
def lr_scheduler(epoch, lr):
    if epoch % 20 == 0:
        return lr * 0.5
    return lr

for epoch in range(100):
    print(f'Start of epoch {epoch+1}')

    # Set learning rate for this epoch
    lr = lr_scheduler(epoch, model.optimizer.learning_rate)
    model.optimizer.learning_rate = lr

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        # Perform forward and backward pass
        logs = model.train_step([x_batch_train, y_batch_train])

        # Log metrics
        if step % 200 == 0:
            print(f'Training loss (for one batch) at step {step}: {logs["loss"]}')
            print(f'Seen so far: {(step + 1) * 64} samples')

    # Save weights every 10 epochs
    if (epoch + 1) % 10 == 0:
        model.save_weights(f'./weights_{epoch+1}.h5')


In [9]:
def lr_scheduler(epoch, lr):
    if epoch % 20 == 0:
        return lr * 0.5
    return lr


In [8]:
optimizer = tf.keras.optimizers.SGD()

2023-06-16 01:07:08.834235: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-16 01:07:08.834721: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [15]:
optimizer.learning_rate

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>